# Compute leadership measures
Calculate leadership measures for each pedestrian in individual networks

## 1. Run *2_get_leadership.py* on Oscar
CI values are too computationally heavy, so run this script on supercomputer for each trial/segment (we used [Oscar](https://docs.ccv.brown.edu/oscar)).

## 2. Compile the trial files

Output:
* `../data/pickle/sayles_leadership.p`

In [ ]:
# first compile trial 7
import pickle

# ===== CHANGE THIS ========================================
ntwk_window_sizes = [0.5]   # , 1
prune_type = 'pruned'       # unpruned or pruned
# ==========================================================

measure_types = ['NI', 'NBI', 'CI', 'CBI']
trial = 7

for ntwk_window_size in ntwk_window_sizes:
    ntwk_window_size_ms = int(ntwk_window_size * 1000)
    weights = pickle.load( open(f'../data/pickle/sayles_weights_pruned_{ntwk_window_size_ms}ms.p', 'rb') )
    segs = weights[trial].keys()

    leadership = {}
    for measure_type in measure_types:
        leadership[measure_type] = {}
        leadership[measure_type+'rank'] = {}

    for seg in segs:
        filename_seg = f'sayles_{prune_type}_trial{trial}_seg{seg}_{ntwk_window_size_ms}ms.p'
        leadership_seg = pickle.load( open(f'../data/sayles_leadership_Oscar/{filename_seg}', 'rb') )

        for measure_type in measure_types:
            leadership[measure_type][seg] = leadership_seg[measure_type][seg]
            leadership[measure_type+'rank'][seg] = leadership_seg[measure_type+'rank'][seg]

    filename_trial = f'sayles_{prune_type}_trial{trial}_{ntwk_window_size_ms}ms.p'
    # e.g., leadership['NIrank'][seg][ntwk,i]
    pickle.dump(leadership, open(f'../data/sayles_leadership_Oscar/{filename_trial}', 'wb'))
    print(filename_trial, 'saved')

In [ ]:
import pickle

# ===== CHANGE THIS ========================================
ntwk_window_sizes = [0.5]   # , 1
prune_type = 'pruned'       # unpruned or pruned
# ==========================================================

measure_types = ['NI', 'NBI', 'CI', 'CBI']

# the size of network time window (in s)
for ntwk_window_size in ntwk_window_sizes:
    print('compiling leadership for', prune_type, 'networks (network window size:', ntwk_window_size, 's)')
    ntwk_window_size_ms = int(ntwk_window_size * 1000)

    leadership = {}
    for measure_type in measure_types:
        leadership[measure_type] = {}
        leadership[measure_type+'rank'] = {}

    for trial in range(1,13):
        filename = f'sayles_{prune_type}_trial{trial}_{ntwk_window_size_ms}ms.p'
        leadership_trial = pickle.load( open(f'../data/sayles_leadership_Oscar/{filename}', 'rb') )

        for measure_type in leadership_trial.keys():
            leadership[measure_type][trial] = leadership_trial[measure_type]

    # e.g., leadership['NIrank'][trial][seg][ntwk,i]
    pickle.dump(leadership, open(f'../data/pickle/sayles_leadership_{prune_type}_{ntwk_window_size_ms}ms.p', 'wb'))

print("saved!")

## 3. Run this section to produce leadership measures in unpruned networks

In [ ]:
## init
import numpy as np
import pickle
import config
from utils.get_leadership import get_NI, get_NBI, get_CI, get_CBI, get_rank

## ====== change ONLY HERE ============
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
measure_types = ['NI', 'NBI']
## ====================================

ntwk_window_size_ms = int(ntwk_window_size * 1000)
weights = pickle.load( open(f'../data/pickle/sayles_weights_unpruned_{ntwk_window_size_ms}ms.p', 'rb') )

print('Computing leadership measure types', measure_types)
print('for unpruned networks', '(network window size:', ntwk_window_size, 's)\n')

leadership = {}

for measure_type in measure_types:
    print(measure_type, end=':')
    leadership[measure_type] = {}
    leadership[measure_type+'rank'] = {}

    if measure_type=='NI':
        get_leadership = get_NI
    elif measure_type=='NBI':
        get_leadership = get_NBI
    elif measure_type=='CI':
        get_leadership = get_CI
    elif measure_type=='CBI':
        get_leadership = get_CBI
    else:
        raise ValueError('measure type error')

    for trial in weights.keys():
        print('trial', int(trial))
        leadership[measure_type][trial] = {}
        leadership[measure_type+'rank'][trial] = {}

        for seg in weights[trial].keys():
            print('-- segment', seg)
            num_networks, N, _ = weights[trial][seg].shape    # (num_networks, N, N)

            leadership[measure_type][trial][seg] = np.zeros((num_networks, N))
            leadership[measure_type+'rank'][trial][seg] = np.zeros((num_networks, N))

            for ntwk in range(num_networks):
                print(ntwk+1, end=' ')
                leadership[measure_type][trial][seg][ntwk,:] = get_leadership(weights[trial][seg][ntwk,:,:], order="ij")  # (N,)
                leadership[measure_type+'rank'][trial][seg][ntwk,:] = get_rank(leadership[measure_type][trial][seg][ntwk,:])  # (N,)

            print(end='\n')
    print(end='\n')

# e.g., leadership['NIrank'][trial][seg][ntwk,i]
pickle.dump(leadership, open(f'../data/pickle/sayles_leadership_unpruned_{ntwk_window_size_ms}ms.p', 'wb'))
print("saved!")

# Plot leadership dynamics
Plot how leadership ranking change over time in each segment.

In [ ]:
# plot dynamics (with lines & dots)

##  init 
import pickle
import pandas as pd

from utils.plot_leadership import plot_leadership_dynamics
import config

# ===== CHANGE THIS ========================================
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
prune_type = 'pruned'       # unpruned or pruned
# ==========================================================

ntwk_window_size_ms = int(ntwk_window_size * 1000)
leadership = pickle.load( open('../data/pickle/sayles_leadership_'+prune_type+'_'
                             +str(ntwk_window_size_ms)+'ms.p', 'rb') )
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')

print("plotting leadership dynamics (" + str(ntwk_window_size) + "s)")

for measure_type in leadership.keys():
    if measure_type[-4:]!='rank':
        continue
    print(measure_type)
    
    for trial in leadership[measure_type].keys():
        print('trial', int(trial), '-- segment', end=' ')
        trial_df = df[(df['trial']==trial)]

        for seg in leadership[measure_type][trial].keys():
            print(seg, end=' ')
            for form in ['png', 'svg']:
                plot_leadership_dynamics(leadership[measure_type][trial][seg], measure_type[:2],
                                         description=f'(Trial {trial} Segment {seg})',
                                         saved=True,
                                         output_folder='../output/sayles/'+form+'/leadership_dynamics_dots/'+measure_type+'_'+prune_type+'_'+str(ntwk_window_size_ms)+'ms/',
                                         output_file='rankings_trial'+str(trial)+'_seg'+str(seg),
                                         output_format=form)
        print(end='\n') 
    
print('saved!')

In [ ]:
# for a specific segment & speficied networks

# plot dynamics (with lines & dots)

##  init 
import pickle
import pandas as pd

from utils.plot_leadership import plot_leadership_dynamics
import config

# ===== CHANGE THIS ========================================
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
prune_type = 'pruned'       # unpruned or pruned

# specify networks to plot
trial = 10 #5
seg = 1 #5
start_ntwk_ID = 13 #11  # plot network IDs between start_network and end_network
end_ntwk_ID = 32 #15
# ==========================================================

ntwk_window_size_ms = int(ntwk_window_size * 1000)
leadership = pickle.load( open('../data/pickle/sayles_leadership_'+prune_type+'_'
                             +str(ntwk_window_size_ms)+'ms.p', 'rb') )
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')

print("plotting leadership dynamics (" + str(ntwk_window_size) + "s)")
print(f'trial {trial} segment {seg}, network {start_ntwk_ID} - {end_ntwk_ID}')

for measure_type in leadership.keys():
    if measure_type[-4:]!='rank':
        continue
    print(measure_type)
    
    trial_df = df[(df['trial']==trial)]

    # leadership[measure_type][trial][seg] : (num_networks, N)
    for form in ['png', 'svg']:
        plot_leadership_dynamics(leadership[measure_type][trial][seg][start_ntwk_ID:end_ntwk_ID+1], measure_type[:2],
                                 description=f'(Trial {trial} Segment {seg})',
                                 saved=True,
                                 output_folder=f'../output/sayles/{form}/leadership_dynamics_dots/{measure_type}_{prune_type}_{ntwk_window_size_ms}ms/publication/',
                                 output_file=f'rankings_trial{trial}_seg{seg}_ntwk{start_ntwk_ID}to{end_ntwk_ID}',
                                 output_format=form)
    
print('saved!')

# Plot rank correlation

Based on *code/plot_rank_correlation.m* from repo *FYP*.
Evaluate rank correlation (Spearman's rho).

## Get rank-order correlation

In [ ]:
import numpy as np
import pandas as pd
import pickle
from scipy.stats import spearmanr

from utils.get_network import get_network_weights
from utils.get_leadership import get_NI, get_rank
import config

## ----- change only here -----------------------------------------------------------------
# the max separation between the correlated networks (in terms of the number of networks apart)
max_apart = 10
# leadership measure to use for calculating leadership ranks
measure_type = 'NI'
## --------------------------------------------------------------------------------

# sample frequency
SAMP_FREQ = config.SAMP_FREQ
# tau
tau = config.TAU
# sizes of network time window to consider
ntwk_window_sizes = [0.5, 1, 2, 3, 4]

## ===== init =====
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
index = pickle.load( open( "../data/pickle/index.p", "rb" ) )
## initialize vars
# store lists of rho to average over (same shape as avg_rho but a list)
ls_rho = [ [ [] for _ in range(max_apart) ] for _ in range(len(ntwk_window_sizes)) ]

## ===== calculate ==========
for trial in index.keys():
    print('trial', int(trial), '-- segment', end=' ')
    trial_df = df[(df['trial']==trial)]

    for seg in index[trial].keys():
        print(seg, end=' ')
        seg_df = trial_df[trial_df['segment']==seg]

        x = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='x').to_numpy()
        y = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='y').to_numpy()
        heading = seg_df.pivot(index='frame_segment', columns='ID', 
                               values='heading').to_numpy()

        for idx, ws in enumerate(ntwk_window_sizes):    # index, window size
            ## ----- get network weights for the segment -----
            # weights: (num_networks, N, N)
            pruned_weights_seg = get_network_weights(index[trial][seg],
                                                     x, y, heading, SAMP_FREQ,
                                                     pruning=True,
                                                     TAU=tau,
                                                     NTWK_WINDOW_SIZE=ws)
            
            ## ----- get NI ranks for the segment -----
            num_networks, N, _ = pruned_weights_seg.shape    # (num_networks, N, N)
            if (ws >= num_networks):
                break
            # unpruned_ranks_seg = np.zeros( (num_networks, N) )
            pruned_ranks_seg = np.zeros( (num_networks, N) )
            # print('(network', end=' ')
            for ntwk in range(num_networks):
                if measure_type == 'NI':
                    get_leadership = get_NI
                else:
                    raise ValueError('measure type error :', measure_type) 
                # NI values & ranks in the given network
                pruned_L_seg = get_leadership(pruned_weights_seg[ntwk,:,:], order="ij")  # NI values: (N,)
                pruned_ranks_seg[ntwk,:] = get_rank(pruned_L_seg)  # (N,)

            ## ----- get rank-order correlation (rho) for all segments -----
            # from calculateRankCorrMat()
            rho, pval = spearmanr(pruned_ranks_seg, axis=1, nan_policy='omit')    # (num_networks, num_networks)

            ## ----- get average rho -----
            # from getAvgRankCorr()
            for ntwks_apart in range(1, min(max_apart+1, num_networks)):     # number of networks apart
                # print(ntwks_apart, 'networks apart')
                mask = np.eye(num_networks, k=ntwks_apart, dtype=bool)
                if isinstance(rho, np.ndarray):  # add only if rho is a matrix
                    ls_rho[idx][ntwks_apart-1].extend(rho[mask])
    print(end='\n')

## ----- get average rho -----
# how many correlations each average rho represents
num_datapoints = np.zeros( (len(ntwk_window_sizes), max_apart) )
# average rho to plot
avg_rho = np.zeros( (len(ntwk_window_sizes), max_apart) )
for ws in range(len(ntwk_window_sizes)):
    for ma in range(max_apart):
        num_datapoints[ws][ma] = len(ls_rho[ws][ma])
        avg_rho[ws,ma] = np.nanmean(ls_rho[ws][ma])

var = {}
var['avg_rho'] = avg_rho    # (len(ntwk_window_sizes), max_apart)
var['num_datapoints'] = num_datapoints  # (len(ntwk_window_sizes), max_apart)

pickle.dump( var, open( "../data/pickle/sayles_avg_rho.p", "wb" ) )
print("saved!")

## Print overall rank-order correlation (average)

In [1]:
import pickle
import numpy as np
from tabulate import tabulate

var = pickle.load( open( "../data/pickle/sayles_avg_rho.p", "rb" ) )
avg_rho = var['avg_rho']     # (len(ntwk_window_sizes), max_apart)
num_datapoints = var['num_datapoints']  # (len(ntwk_window_sizes), max_apart)

num_window_sizes, max_apart = avg_rho.shape
x = np.arange(1, max_apart+1)

# sizes of network time window to consider
ntwk_window_sizes = [0.5, 1, 2, 3, 4]

data = []
for i in range(num_window_sizes):
    data.append([ntwk_window_sizes[i]] + list(avg_rho[i,:]))

head = ["Window size"]
for i in range(1, max_apart+1):
    head.append(str(i) + " apart")
 
print(tabulate(data, headers=head, tablefmt="grid"))

+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+
|   Window size |   1 apart |   2 apart |   3 apart |   4 apart |   5 apart |   6 apart |   7 apart |   8 apart |   9 apart |   10 apart |
+===============+===========+===========+===========+===========+===========+===========+===========+===========+===========+============+
|           0.5 |  0.892584 |  0.819012 |  0.763372 |  0.717933 |  0.680128 |  0.646641 |  0.617061 |  0.590499 |  0.567404 |   0.54491  |
+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+
|           1   |  0.843687 |  0.739546 |  0.666895 |  0.608314 |  0.560875 |  0.513802 |  0.466821 |  0.422876 |  0.375296 |   0.337593 |
+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+
|           2   |  0.784592

## Plot overall rank-order correlation (average)

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os

var = pickle.load( open( "../data/pickle/sayles_avg_rho.p", "rb" ) )
avg_rho = var['avg_rho']     # (len(ntwk_window_sizes), max_apart)
num_datapoints = var['num_datapoints']  # (len(ntwk_window_sizes), max_apart)

num_window_sizes, max_apart = avg_rho.shape
x = np.arange(1, max_apart+1)

# sizes of network time window to consider
ntwk_window_sizes = [0.5, 1, 2, 3, 4]

## ===== plot avg_rho as a function on time =====
_, ax = plt.subplots(figsize=(10, 6))
for i in range(num_window_sizes):
    ax.plot(x * ntwk_window_sizes[i], avg_rho[i,:], 
            label=f'{ntwk_window_sizes[i]} seconds', marker='o')

ax.set_xlabel('Seconds apart', fontsize=15)
ax.set_ylabel(r'Average rank correlation ($\rho$)', fontsize=15)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# ax.set_title('Average rank correlation vs. time apart : overall', fontsize=18)
ax.set_xlim(0, 10)  # max
ax.set_ylim(0, 1)
plt.grid()
plt.tight_layout()  # Adjust layout to fit legend

for form in ['png', 'svg']:
    new_folder = f'../output/sayles/{form}/rank_correlation/'
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    plt.savefig(os.path.join(new_folder, 'rank_correlation_time.' + form))
# plt.show()
plt.clf()
plt.close()

# Plot spatial heat maps

## Get values needed for plotting spatial heat maps
Collect leadership values & corresponding positions

In [1]:
import numpy as np
import pickle
import pandas as pd
import config
import warnings
import pickle

from utils.plot_spatial_heatmap import *

# ===== CHANGE THIS ===================================
# specify grid width (m or SD) in spatial heatmaps
grid_width = {}
grid_width['m'] = 0.5
grid_width['SD'] = 0.25
# network window size
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
# prune_type = 'pruned'       # unpruned or pruned

# prob_threshold = 0.005
prob_threshold = {}
prob_threshold['N16'] = 0.005  # has more outliers
prob_threshold['N20'] = 0.002;  prob_threshold['N10'] = 0.002
# =====================================================

In [ ]:
# ===== INIT =====
# --- get data ---
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
groups = ['N16', 'N20', 'N10']
conditions = ['overall'] + groups
units = ['m', 'SD']
axes = ['x', 'y']
prune_types = ['unpruned', 'pruned']

ntwk_window_size_ms = int(ntwk_window_size * 1000)
SAMP_FREQ = config.SAMP_FREQ
leadership = {}
for prune_type in prune_types:
    # leadership[prune_type][measure_type][trial][seg] : (num_networks, N)
    leadership[prune_type] = pickle.load( open(f'../data/pickle/sayles_leadership_{prune_type}_{ntwk_window_size_ms}ms.p', 'rb') )
num_frames_network = int(SAMP_FREQ * ntwk_window_size)

# --- initialize variables ---
# positions[cond][ax][unit] : a python list for positions 
# shape : (num_datapoints,)
positions = {}
for cond in conditions:
    positions[cond] = {}
    for ax in axes: 
        positions[cond][ax] = {}
        for unit in units:
            positions[cond][ax][unit] = []     # values will be added to this list
# values[cond][prune_type][measure_type] : a python list of leadership value
# shape : (num_datapoints,)
values = {}
for cond in conditions:
    values[cond] = {}
    for prune_type in prune_types:
        values[cond][prune_type] = {}
        for measure_type in leadership[prune_type].keys():
            values[cond][prune_type][measure_type] = []     # values will be added to this list
temp_all_x_m, temp_all_y_m = [], []
# PDF[cond][unit] : a numpy array of leadership value
# shape : (num_y_cells, num_x_cells) -> this gets rotated when plotting
PDF = {}
for cond in conditions:
    PDF[cond] = {}
    for unit in units:
        PDF[cond][unit] = []    # this list will be replaced with np array
# min/max positions on X & Y axes
lims = ['xmin', 'xmax', 'ymin', 'ymax']
position_limits = {}
for unit in units:
    position_limits[unit] = {}
    for lim in lims:
        position_limits[unit][lim] = float('inf') if (lim[1:]=='min') else float('-inf')

# ===== collect positions (in meters) & leadership values per group =====
temp_prune = prune_types[0]
temp_measure = list(leadership[prune_types[0]].keys())[0]   # this is only used to access general variable shapes

for trial in leadership[temp_prune][temp_measure].keys():
    for seg in leadership[temp_prune][temp_measure][trial].keys():
        num_networks, N = leadership[temp_prune][temp_measure][trial][seg].shape
        group = 'N' + str(N)

        ## ----- get positions (meters) -----
        seg_df = df[(df['trial']==trial) & (df['segment']==seg)]
        # get node positions (absolute; in m) in the given segment
        x_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                               values='x_transformed').to_numpy()
        y_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                        values='y_transformed').to_numpy()
        # get node positions (absolute; in m) in the 1st frame of each time window per network
        x_M = x_M_seg[::num_frames_network, :][:num_networks,:].flatten()   # (num_networks, N) -> flattened
        y_M = y_M_seg[::num_frames_network, :][:num_networks,:].flatten()   # (num_networks, N) -> flattened
        # get a mask
        temp_value = leadership[temp_prune][temp_measure][trial][seg].flatten()      # (num_networks, N) -> flattened
        mask = ~np.isnan(temp_value) * ~np.isnan(x_M) * ~np.isnan(y_M)   # look for nan
        # add positions to lists
        positions[group]['x']['m'] += list(x_M[mask])
        positions[group]['y']['m'] += list(y_M[mask])
        # add to temporary lists; used to calculate position_limits
        temp_all_x_m += list(x_M[mask])   
        temp_all_y_m += list(y_M[mask])

        ## ----- get leadership values  -----
        for prune_type in prune_types:
            for measure_type in leadership[prune_type].keys():  # get leadership values for each measure type
                v = leadership[prune_type][measure_type][trial][seg].flatten()    # (num_networks, N) -> flattened
                if (measure_type[-4:] == "rank"):  # flip ranking (higher value = higher rank)
                    v = (N + 1 - v) / N
                # add leadership values to lists
                values[group][prune_type][measure_type] += list(v[mask])

# ===== calculate PDF per group & remove outlier datapoints =====
total_datapoints = 0
filtered_datapoints = 0
print('filtering out outlier datapoints...')

def count_data_points(x, y, 
                      xmin, xmax, ymin, ymax,
                      grid_width=0.5,
                      threshold=5):
    x_edges = np.arange(xmin, xmax + grid_width, grid_width)
    y_edges = np.arange(ymin, ymax + grid_width, grid_width)
    counts, _, _ = np.histogram2d(x, y, bins=[x_edges, y_edges])
    return counts < threshold

for group in groups:
    # --- find outliers based on cell probability threshold ---
    mask = count_data_points(positions[group]['x']['m'], 
                             positions[group]['y']['m'], 
                             int(np.floor(np.min(positions[group]['x']['m']))), 
                             int(np.ceil(np.max(positions[group]['x']['m']))),
                             int(np.floor(np.min(positions[group]['y']['m']))), 
                             int(np.ceil(np.max(positions[group]['y']['m']))),
                             grid_width=grid_width['m'])
    print(mask)
    # --- count filtered datapoints ---
    print(f'{group} (probability threshold = {prob_threshold[group]}) : ', end='')
    print(f'{round((~mask).sum()/len(mask)*100, 2)}% of datapoints removed (total: {len(mask)}, filtered: {(~mask).sum()})')
    # print(f"{group}  :", end=" ")
    total_datapoints += len(mask)
    filtered_datapoints += (~mask).sum()
    # --- update xy poitions (in m) & leadership (filter outliers out) ---
    positions[group]['x']['m'] = list(np.array(positions[group]['x']['m'])[mask])
    positions[group]['y']['m'] = list(np.array(positions[group]['y']['m'])[mask])
    for prune_type in prune_types:
        for measure_type in leadership[prune_type].keys():
            values[group][prune_type][measure_type] = list(np.array(values[group][prune_type][measure_type])[mask])
    positions['overall']['x']['m'] += positions[group]['x']['m']
    positions['overall']['y']['m'] += positions[group]['y']['m']

print(f'all groups: {round(filtered_datapoints/total_datapoints*100, 2)}% of datapoints removed (total: {total_datapoints}, filtered: {filtered_datapoints})\n')
position_limits['m']['xmin'] = int(np.floor(np.min(positions['overall']['x']['m'])))
position_limits['m']['xmax'] = int(np.ceil(np.max(positions['overall']['x']['m'])))
position_limits['m']['ymin'] = int(np.floor(np.min(positions['overall']['y']['m'])))
position_limits['m']['ymax'] = int(np.ceil(np.max(positions['overall']['y']['m'])))

# ===== calculate PDF & get SDs per group =====
print('SDs on x & y axes per group...')
for group in groups:
    # --- recalculate PDF to get SD per group ---
    SD_width_x, SD_width_y, PDF[group]['m'] = get_SD(positions[group]['x']['m'], positions[group]['y']['m'],
                                                     xmin=position_limits['m']['xmin'], 
                                                     xmax=position_limits['m']['xmax'], 
                                                     ymin=position_limits['m']['ymin'], 
                                                     ymax=position_limits['m']['ymax'], 
                                                     grid_width=grid_width['m'])
    print(group, ":", end=" ")
    print(SD_width_x, SD_width_y)
    # --- transform the coordinates (m -> SD) ---
    temp_x_SD, temp_y_SD = transform_m2SD(positions[group]['x']['m'], positions[group]['y']['m'], 
                                          SD_width_x, SD_width_y)
    positions[group]['x']['SD'] = list(temp_x_SD)
    positions[group]['y']['SD'] = list(temp_y_SD)
    
# ===== combine all groups for overall heat mapss =====
for group in groups:
    # add position data
    for ax in axes: 
        # for unit in units:
        positions['overall'][ax]['SD'] += positions[group][ax]['SD']
    # add leadership values
    for prune_type in prune_types:
        for measure_type in leadership[prune_type].keys():
            values['overall'][prune_type][measure_type] += values[group][prune_type][measure_type]


# ===== collect overall position limits (in m & SD) =====
print("\nposition limits (rounded)...")
position_limits['SD']['xmin'] = int(np.floor(np.min(positions['overall']['x']['SD'])))
position_limits['SD']['xmax'] = int(np.ceil(np.max(positions['overall']['x']['SD'])))
position_limits['SD']['ymin'] = int(np.floor(np.min(positions['overall']['y']['SD'])))
position_limits['SD']['ymax'] = int(np.ceil(np.max(positions['overall']['y']['SD'])))

# ===== get probability density =====
for unit in units:
    PDF['overall'][unit] = get_PDF(positions['overall']['x'][unit], positions['overall']['y'][unit],
                                   xmin=position_limits[unit]['xmin'], xmax=position_limits[unit]['xmax'], 
                                   ymin=position_limits[unit]['ymin'], ymax=position_limits[unit]['ymax'], 
                                   grid_width=grid_width[unit])
for group in groups:
    PDF[group]['SD'] = get_PDF(positions[group]['x']['SD'], positions[group]['y']['SD'], 
                               xmin=position_limits['SD']['xmin'], xmax=position_limits['SD']['xmax'], 
                               ymin=position_limits['SD']['ymin'], ymax=position_limits['SD']['ymax'], 
                               grid_width=grid_width['SD'])

for unit in units:
    print(unit, list(position_limits[unit].keys()), ":", end=" ")
    print(position_limits[unit]['xmin'],
          position_limits[unit]['xmax'],
          position_limits[unit]['ymin'],
          position_limits[unit]['ymax'] )

# ===== check shapes =====
# for cond in conditions:
print('checking matrix shapes...')
for cond in ['overall']:
    print(cond)
    for unit in units:
        print("---", unit, ":")
        print("   ", PDF[cond][unit].shape)

print("\nDone")

## Calculate average leadership values for each cell in spatial heat maps

In [ ]:
## ===== init =====
# specify bins on both axes (e.g., [-3, 4]: 0.5 step)
x_range, y_range = {}, {}
for unit in units:
    x_range[unit] = list(np.arange(position_limits[unit]['xmin'], 
                                   position_limits[unit]['xmax'] + grid_width[unit], 
                                   grid_width[unit]))     
    y_range[unit] = list(np.arange(position_limits[unit]['ymax'], 
                                   position_limits[unit]['ymin'] - grid_width[unit], 
                                   -grid_width[unit]))

output_leadership_avg = {}

# some cells with no data end up being a empty list, which gives "RuntimeWarning: Mean of empty slice"
# but this can be ignored
warnings.filterwarnings("ignore", category=RuntimeWarning) 

## ===== calculate average leadership in each cell =====
for cond in conditions:
    # print(cond)
    output_leadership_avg[cond] = {}

    for unit in units:
        # print("---", unit)
        output_leadership_avg[cond][unit] = {}
        # get cell indices for each datapoint (num_x_values = num_y_values)
        x_digitized = np.digitize(positions[cond]['x'][unit], np.array(x_range[unit]))
        y_digitized = np.digitize(positions[cond]['y'][unit], np.array(y_range[unit]))

        # get leadership value for each data and add to the list of corresponding cell
        for prune_type in prune_types:
            # print("------", prune_type, ":", end=" ")
            output_leadership_avg[cond][unit][prune_type] = {}
            
            for measure_type in leadership[prune_type].keys():
                # print(measure_type, end=" ")
                # output_leadership_avg[cond][unit][measure_type] = [[[] for _ in range(len(y_range))] for _ in range(len(x_range))]
                temp_ls = [[[] for _ in range(len(y_range[unit]))] for _ in range(len(x_range[unit]))]
                # collect values for each cell
                for d in range(x_digitized.shape[0]):  # for each datapoint
                    temp_ls[x_digitized[d]][y_digitized[d]].append(values[cond][prune_type][measure_type][d])
                # calculate the leadership average in each cell
                output_leadership_avg[cond][unit][prune_type][measure_type] = np.zeros((len(x_range[unit]), len(y_range[unit])))
                for x_ind in range(len(x_range[unit])):
                    for y_ind in range(len(y_range[unit])):
                        output_leadership_avg[cond][unit][prune_type][measure_type][x_ind, y_ind] = np.nanmean(temp_ls[x_ind][y_ind])
            # print()

# ===== check shapes =====
isError = False
for cond in conditions:
    for unit in units:
        for prune_type in prune_types:
            for measure_type in leadership[prune_type].keys():
                if (PDF[cond][unit].shape != output_leadership_avg[cond][unit][prune_type][measure_type].shape):
                    isError = True
                    print("error:", cond, unit, prune_type, measure_type)
                    print("PDF shape:", PDF[cond][unit].shape)
                    print("output_leadership_avg shape:", output_leadership_avg[cond][unit][prune_type][measure_type].shape)
                    
if not isError:
    print("no error")

## Plot spatial heat maps

In [ ]:
print('plotting...')

# ===== change this =====
saved = True
# =======================

for cond in conditions:
    print(cond, end=' : ')
    for unit in units:
        print(unit, end=' ')
        # plot PDF
        for form in ['png', 'svg']:
            plot_PDF(PDF[cond][unit],
                     position_limits[unit]["xmin"], 
                     position_limits[unit]["xmax"], 
                     position_limits[unit]["ymin"], 
                     position_limits[unit]["ymax"],
                     unit=unit,
                     cmap_name="jet",
                     grid_width=grid_width[unit],
                     saved=saved,
                     plotGaussian=True, 
                     output_folder=f'../output/sayles/{form}/spatial_heatmaps/PDF/{unit}_{ntwk_window_size_ms}ms/',
                     output_file=f'sayles_{unit}_{cond}',
                     output_format=form)
        for prune_type in prune_types:
            for measure_type in leadership[prune_type].keys():
                for form in ['png', 'svg']:
                    # plot leadership
                    plot_leadership_heatmap(output_leadership_avg[cond][unit][prune_type][measure_type], 
                                            position_limits[unit]["xmin"], position_limits[unit]["xmax"], 
                                            position_limits[unit]["ymin"], position_limits[unit]["ymax"],
                                            unit=unit,
                                            grid_width=grid_width[unit],
                                            saved=saved,
                                            title=measure_type,
                                            output_folder=f'../output/sayles/{form}/spatial_heatmaps/leadership/{unit}_{prune_type}_{ntwk_window_size_ms}ms/',
                                            output_file=f'sayles_{measure_type}_{unit}_{cond}',
                                            output_format=form)
    print()
                
print('done!')

# Plot individual leadership index

# Plot the relationship between postion & individual leadership

In [ ]:
import numpy as np
import pickle
import pandas as pd
import config
import warnings
import pickle
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.metrics import r2_score

from utils.plot_spatial_heatmap import *
from utils.plot_leadership import plot_confidence_ellipse

# ===== CHANGE THIS ===================================
# specify grid width (m or SD) in spatial heatmaps
# kde_grid_width = 0.5
grid_width = {}
grid_width['m'] = 0.5
grid_width['SD'] = 0.25
# network window size
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
prune_type = 'pruned'       # unpruned or pruned
n_std = 1   # radius (SD) of confidence ellipse

# prob_threshold = 0.005
prob_threshold = {}
prob_threshold['N16'] = 0.002;  prob_threshold['N20'] = 0.005;  prob_threshold['N10'] = 0.005
# =====================================================

In [ ]:
# to plot SD-based position values

# ===== INIT =====
# --- get data ---
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
groups = ['N16', 'N20', 'N10']
conditions = ['overall'] + groups
units = ['m', 'SD']
axes = ['x', 'y']

ntwk_window_size_ms = int(ntwk_window_size * 1000)
SAMP_FREQ = config.SAMP_FREQ
leadership = {}
leadership = pickle.load( open('../data/pickle/sayles_leadership_' + prune_type + '_' + str(ntwk_window_size_ms) + 'ms.p', 'rb') )
num_frames_network = int(SAMP_FREQ * ntwk_window_size)
measure_types = leadership.keys()

# --- initialize variables ---
# positions[cond][ax][unit] : a python list for positions 
# shape : (num_datapoints,)
positions = {}
for cond in conditions:
    positions[cond] = {}
    for ax in axes: 
        positions[cond][ax] = {}
        for unit in units:
            positions[cond][ax][unit] = []     # values will be added to this list
temp_all_x_m, temp_all_y_m = [], []
# min/max positions on X & Y axes
lims = ['xmin', 'xmax', 'ymin', 'ymax']
position_limits = {}
for unit in units:
    position_limits[unit] = {}
    for lim in lims:
        position_limits[unit][lim] = float('inf') if (lim[1:]=='min') else float('-inf')

# ===== collect positions (in meters) & leadership values per group =====
temp_measure = list(leadership.keys())[0]   # this is only used to access general variable shapes

for trial in leadership[temp_measure].keys():
    for seg in leadership[temp_measure][trial].keys():
        num_networks, N = leadership[temp_measure][trial][seg].shape
        group = 'N' + str(N)

        ## ----- get positions (meters) -----
        seg_df = df[(df['trial']==trial) & (df['segment']==seg)]
        # get node positions (absolute; in m) in the given segment
        x_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                               values='x_transformed').to_numpy()
        y_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                        values='y_transformed').to_numpy()
        # get node positions (absolute; in m) in the 1st frame of each time window per network
        x_M = x_M_seg[::num_frames_network, :][:num_networks,:].flatten()   # (num_networks, N) -> flattened
        y_M = y_M_seg[::num_frames_network, :][:num_networks,:].flatten()   # (num_networks, N) -> flattened
        # get a mask
        temp_value = leadership[temp_measure][trial][seg].flatten()      # (num_networks, N) -> flattened
        mask = ~np.isnan(temp_value) * ~np.isnan(x_M) * ~np.isnan(y_M)   # look for nan
        # add positions to lists
        positions[group]['x']['m'] += list(x_M[mask])
        positions[group]['y']['m'] += list(y_M[mask])
        # add to temporary lists; used to calculate position_limits
        temp_all_x_m += list(x_M[mask])   
        temp_all_y_m += list(y_M[mask])

# ===== calculate PDF per group =====
for group in groups:
    # --- find outliers based on cell probability threshold ---
    mask = get_filter(positions[group]['x']['m'], positions[group]['y']['m'],
                      xmin=int(np.floor(np.min(temp_all_x_m))), 
                      xmax=int(np.ceil(np.max(temp_all_x_m))),
                      ymin=int(np.floor(np.min(temp_all_y_m))), 
                      ymax=int(np.ceil(np.max(temp_all_y_m))),
                      grid_width=grid_width['m'],
                      threshold=prob_threshold[group])
    # --- update xy poitions (in m) & leadership (filter outliers out) ---
    positions[group]['x']['m'] = list(np.array(positions[group]['x']['m'])[mask])
    positions[group]['y']['m'] = list(np.array(positions[group]['y']['m'])[mask])
    positions['overall']['x']['m'] += positions[group]['x']['m']
    positions['overall']['y']['m'] += positions[group]['y']['m']

position_limits['m']['xmin'] = int(np.floor(np.min(positions['overall']['x']['m'])))
position_limits['m']['xmax'] = int(np.ceil(np.max(positions['overall']['x']['m'])))
position_limits['m']['ymin'] = int(np.floor(np.min(positions['overall']['y']['m'])))
position_limits['m']['ymax'] = int(np.ceil(np.max(positions['overall']['y']['m'])))

SD_width_x, SD_width_y = {}, {}
for group in groups:
    # --- recalculate PDF to get SD per group ---
    SD_width_x[group], SD_width_y[group], _ = get_SD(positions[group]['x']['m'], positions[group]['y']['m'],
                                                     xmin=position_limits['m']['xmin'], xmax=position_limits['m']['xmax'], 
                                                     ymin=position_limits['m']['ymin'], ymax=position_limits['m']['ymax'], 
                                                     grid_width=grid_width['m'])
    print(f"{group} (probability threshold = {prob_threshold[group]}) :", end=" ")
    print(SD_width_x[group], SD_width_y[group])

In [ ]:
output = {}
for trial in leadership[list(measure_types)[0]].keys():
    for seg in leadership[list(measure_types)[0]][trial].keys():
        num_networks, N = leadership[list(measure_types)[0]][trial][seg].shape
        cond = 'N'+str(N)
        if cond not in output.keys():
            output[cond] = {}

        ## ----- positions (meters) -----
        seg_df = df[(df['trial']==trial) & (df['segment']==seg)]
        # get node positions (absolute; in m) in the given segment
        x_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                        values='x_transformed').to_numpy()     # values='x'
        y_M_seg = seg_df.pivot(index='frame_segment', columns='ID', 
                        values='y_transformed').to_numpy()     # values='y'
        # get node positions (absolute; in m) in the 1st frame of each time window per network
        x_M = x_M_seg[::num_frames_network, :][:num_networks,:]   # (num_networks, N)
        y_M = y_M_seg[::num_frames_network, :][:num_networks,:]   # (num_networks, N)
        x_SD, y_SD = transform_m2SD(x_M, y_M, SD_width_x[cond], SD_width_y[cond])   # (num_networks, N)
        
        for ped in range(N):
            # get a mask
            temp_value = leadership[list(measure_types)[0]][trial][seg][:,ped]     # (num_networks, N)
            mask = ~np.isnan(temp_value) * ~np.isnan(x_SD[:,ped]) * ~np.isnan(y_SD[:,ped])   # look for nan

            if ped not in output[cond].keys():
                output[cond][ped] = {}
                output[cond][ped]['x_m'] = list(x_M[:,ped][mask])
                output[cond][ped]['y_m'] = list(y_M[:,ped][mask])
                output[cond][ped]['x_SD'] = list(x_SD[:,ped][mask])
                output[cond][ped]['y_SD'] = list(y_SD[:,ped][mask])
                for measure_type in measure_types:
                    output[cond][ped][measure_type] = list(leadership[measure_type][trial][seg][:,ped][mask])
            else:
                output[cond][ped]['x_m'] += list(x_M[:,ped][mask])
                output[cond][ped]['y_m'] += list(y_M[:,ped][mask])
                output[cond][ped]['x_SD'] += list(x_SD[:,ped][mask])
                output[cond][ped]['y_SD'] += list(y_SD[:,ped][mask])
                for measure_type in measure_types:
                    output[cond][ped][measure_type] += list(leadership[measure_type][trial][seg][:,ped][mask])

# normalize ranking
for measure_type in measure_types:
    if measure_type[2:] != 'rank':
        continue

    for cond in output.keys():
        N = int(cond[1:])
        for ped in output[cond].keys():
            output[cond][ped][measure_type] = np.array(output[cond][ped][measure_type]) / N

In [ ]:
# run this to plot all groups together

cond_colors = ['tab:blue', 'tab:orange', 'tab:green']
cond_markers = ["o", "x", "^"]

for measure_type in measure_types:
    print(measure_type)

    for unit in units:
        print('--', unit)
        for plot_type in ["mean_only", "confidence_ellipse", "scatter"]:    # 0: plot scatter, 1: plot confidence ellipse
            # print('----', plot_type)
            ver_min, ver_max = [-1.1, 1.1]
            if plot_type=="scatter":
                hor_min, hor_max = [-2, 2]
            else:   # "scatter"
                # hor_min, hor_max = [-3, 4]
                hor_min, hor_max = [-3, 3]
                
            # Plot
            _, ax = plt.subplots(figsize=(10, 6))
            all_hor_mean, all_ver_mean = [], []     # (total_N,)

            # for i, cond in enumerate(output.keys()):
            for i, cond in enumerate(['N10', 'N16', 'N20']):
                cond_hor_mean, cond_ver_mean = [], []   # (N,)

                for ped in output[cond].keys():
                    if plot_type=="confidence_ellipse":
                        plot_confidence_ellipse(np.array(output[cond][ped]['y_'+unit]), 
                                                np.array(output[cond][ped][measure_type]), 
                                                ax, n_std=n_std, edgecolor='black',
                                                alpha=0.2)
                    elif plot_type=="scatter":
                        ax.scatter(np.array(output[cond][ped]['y_'+unit]), 
                                    np.array(output[cond][ped][measure_type]))
                    # calculate nanmean of y and measure_type for the given ped
                    cond_hor_mean.append(np.nanmean(output[cond][ped]['y_'+unit]))
                    cond_ver_mean.append(np.nanmean(output[cond][ped][measure_type]))
                # add to lists of all datapoints
                all_hor_mean += cond_hor_mean
                all_ver_mean += cond_ver_mean
                # change to numpy arrays
                cond_hor_mean = np.array(cond_hor_mean)
                cond_ver_mean = np.array(cond_ver_mean)
                # plot means in the group 
                ax.scatter(cond_hor_mean, cond_ver_mean, 
                           color=cond_colors[i],
                           marker=cond_markers[i],
                           label=cond)
                # print r_squared for the group
                m, b = np.polyfit(cond_hor_mean, cond_ver_mean, 1)
                r_squared = r2_score(cond_ver_mean, m * cond_hor_mean + b)
                if (plot_type=="confidence_ellipse"):
                    print('    ', cond, ':', f'y = {m:.2f}x {b:+.2f}, R^2 = {r_squared:.2f}')
            
            all_hor_mean = np.array(all_hor_mean)
            all_ver_mean = np.array(all_ver_mean)

            # obtain m (slope) and b(intercept) of linear regression line
            m, b = np.polyfit(all_hor_mean, all_ver_mean, 1)
            r_squared = r2_score(all_ver_mean, m * all_hor_mean + b)
            plt.axline(xy1=(0, b), slope=m, 
                       label=f'$y = {m:.2f}x {b:+.2f}$, $R^2 = {r_squared:.2f}$',
                       color='red')
            if (plot_type=="confidence_ellipse"):
                print('     overall :', f'y = {m:.2f}x {b:+.2f}, R^2 = {r_squared:.2f}')
            
            ax.set_xlim(hor_min, hor_max)
            ax.set_ylim(ver_min, ver_max)
            plt.xlabel('Back - Front (Mean Y axis; ' + unit + ')', fontsize=15)
            plt.ylabel('Mean ' + measure_type + ' value', fontsize=15)
            plt.legend()
            # plt.title('Mean ' + measure_type + ' as a function of mean Y axis (' + unit + ') - overall', fontsize=18)

            for form in ['png', 'svg']:
                new_folder = '../output/sayles/'+form+'/individual_leadership/' + plot_type + '/' + unit + '/'
                if not os.path.exists(new_folder):
                    os.makedirs(new_folder)
                plt.savefig(os.path.join(new_folder, 'individual_leadership_' + measure_type + '_overall.' + form))
            # plt.show()
            plt.clf()
            plt.close()

    print(end='\n')
    
print('done')

In [ ]:
# # run this to plot individual group separately

# for measure_type in measure_types:
#     print(measure_type, end=' : ')
#     for cond in output.keys():
#         print(cond, end=' ')

#         for unit in units:
#             for plot_type in [0,1]:    # 0: plot scatter, 1: plot confidence ellipse
#                 # Plot
#                 _, ax = plt.subplots(figsize=(10, 6))
#                 ls_hor_mean, ls_ver_mean = [], []

#                 ver_min, ver_max = [-1.1, 1.1]
#                 if plot_type:
#                     hor_min, hor_max = [-3, 4]
#                 else:
#                     hor_min, hor_max = [-2, 2]

#                 for ped in output[cond].keys():
#                     if plot_type:
#                         plot_confidence_ellipse(np.array(output[cond][ped]['y_'+unit]), 
#                                                 np.array(output[cond][ped][measure_type]), 
#                                                 ax, n_std=n_std, edgecolor='black')
#                     else:
#                         ax.scatter(np.array(output[cond][ped]['y_'+unit]), 
#                                     np.array(output[cond][ped][measure_type]))

#                     # Calculate nanmean of y and measure_type
#                     ls_hor_mean.append(np.nanmean(output[cond][ped]['y_'+unit]))
#                     ls_ver_mean.append(np.nanmean(output[cond][ped][measure_type]))

#                 ls_hor_mean = np.array(ls_hor_mean)     # (total_N,)
#                 ls_ver_mean = np.array(ls_ver_mean)     # (total_N,)
#                 # obtain m (slope) and b(intercept) of linear regression line
#                 m, b = np.polyfit(ls_hor_mean, ls_ver_mean, 1)
#                 r_squared = r2_score(ls_ver_mean, m * ls_hor_mean + b)
#                 plt.axline(xy1=(0, b), slope=m, 
#                         #    label=f'$y = {m:.1f}x {b:+.1f}$',
#                         label=f'$y = {m:.1f}x {b:+.1f}$, $R^2 = {r_squared:.2f}$',
#                         color='red')
#                 plt.scatter(ls_hor_mean, ls_ver_mean, color='blue')

#                 ax.set_xlim(hor_min, hor_max)
#                 ax.set_ylim(ver_min, ver_max)
#                 plt.xlabel('Back - Front (Mean Y axis; ' + unit + ')')
#                 plt.ylabel('Mean ' + measure_type + ' value')
#                 plt.legend()
#                 plt.title('Mean ' + measure_type + ' as a function of mean Y axis (' + unit + ') - ' + cond)

#                 for form in ['png', 'svg']:
#                     new_folder = '../output/sayles/'+form+'/individual_leadership/'
#                     if plot_confidence_ellipse:
#                         new_folder += 'confidence_ellipse/'
#                     else:
#                         new_folder += 'scatter/'
#                     if not os.path.exists(new_folder):
#                         os.makedirs(new_folder)
#                     plt.savefig(os.path.join(new_folder, 'individual_leadership_' + measure_type + '_' + cond + '_' + unit + '.' + form))
#                 # plt.show()
#                 plt.clf()
#                 plt.close()

#     print(end='\n')
    
# print('done')

In [ ]:
# get stats

import warnings
import statsmodels.api as sm

warnings.filterwarnings("ignore", category=RuntimeWarning) 

for measure_type in measure_types:
    print(measure_type)

    for unit in ['SD']:
        print('--', unit)
        for plot_type in ["confidence_ellipse", "scatter"]:    # 0: plot scatter, 1: plot confidence ellipse
            # print('----', plot_type)

            all_hor_mean, all_ver_mean = [], []     # (total_N,)

            # for i, cond in enumerate(output.keys()):
            for i, cond in enumerate(['N10', 'N16', 'N20']):
                cond_hor_mean, cond_ver_mean = [], []   # (N,)

                for ped in output[cond].keys():
                    # calculate nanmean of y and measure_type for the given ped
                    cond_hor_mean.append(np.nanmean(output[cond][ped]['y_'+unit]))
                    cond_ver_mean.append(np.nanmean(output[cond][ped][measure_type]))
                # add to lists of all datapoints
                all_hor_mean += cond_hor_mean
                all_ver_mean += cond_ver_mean
                # change to numpy arrays
                cond_hor_mean = np.array(cond_hor_mean)
                cond_ver_mean = np.array(cond_ver_mean)

                # # print r_squared for the group
                # m, b = np.polyfit(cond_hor_mean, cond_ver_mean, 1)
                # r_squared = r2_score(cond_ver_mean, m * cond_hor_mean + b)
                # if (plot_type=="confidence_ellipse"):
                #     print('    ', cond, ':', f'y = {m:.1f}x {b:+.1f}, R^2 = {r_squared:.2f}')

                #add constant to predictor variables
                cond_hor_mean = sm.add_constant(cond_hor_mean)
                #fit linear regression model
                model = sm.OLS(cond_ver_mean, cond_hor_mean).fit()
                print(cond)
                print(model.summary())
            
            all_hor_mean = np.array(all_hor_mean)
            all_ver_mean = np.array(all_ver_mean)

            # # obtain m (slope) and b(intercept) of linear regression line
            # m, b = np.polyfit(all_hor_mean, all_ver_mean, 1)
            # r_squared = r2_score(all_ver_mean, m * all_hor_mean + b)
            # if (plot_type=="confidence_ellipse"):
            #     print('     overall :', f'y = {m:.1f}x {b:+.1f}, R^2 = {r_squared:.2f}')

            #add constant to predictor variables
            all_hor_mean = sm.add_constant(all_hor_mean)
            #fit linear regression model
            model = sm.OLS(all_ver_mean, all_hor_mean).fit()
            print('overall')
            print(model.summary())